In [21]:
# KV‑cache compression analysis (open‑source only)
# ------------------------------------------------
import json, glob, re, statistics, itertools, os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# Ensure the plots directory exists
plots_dir = "./plots"
os.makedirs(plots_dir, exist_ok=True)

# 1) locate result files in the current directory
files = glob.glob("benchmark_results_distilgpt2_*.json")
if not files:
    raise FileNotFoundError("No benchmark_results_distilgpt2_*.json files in this folder!")

records: List[Dict[str, Any]] = []

# Function to extract latent dimension from filename
def get_latent_dim(filename: str) -> Optional[int]:
    match_ld = re.search(r"distilgpt2_(\d+)\.json$", filename)
    if match_ld:
        return int(match_ld.group(1))
    return None

for path in files:
    latent_dim = get_latent_dim(path)
    if latent_dim is not None:
        with open(path) as fp:
            res = json.load(fp)

        raw_ppl_wiki = res.get("raw_baseline_ppl")
        ae_ppl_wiki_none = res["perplexities"].get("none", {}).get("ae_compressed_ppl")

        if raw_ppl_wiki is not None and ae_ppl_wiki_none is not None:
            records.append(dict(
                latent_dim=latent_dim,
                raw_ppl=raw_ppl_wiki,
                ae_ppl=ae_ppl_wiki_none,
                dataset="WikiText"
            ))

        lb_baseline = res.get("longbench", {}).get("baseline", {})
        lb_compressed_none = res.get("longbench", {}).get("compressed", {}).get("none", {})

        for task, base_ppl in lb_baseline.items():
            comp_ppl = lb_compressed_none.get(task)
            if comp_ppl is not None:
                records.append(dict(
                    latent_dim=latent_dim,
                    raw_ppl=base_ppl,
                    ae_ppl=comp_ppl,
                    dataset=f"LongBench ({task})"
                ))

df = pd.DataFrame(records).sort_values(["latent_dim", "dataset"]).reset_index(drop=True)

latent_dims = sorted(df['latent_dim'].dropna().unique())
longbench_tasks = sorted([d.split('(')[1][:-1] for d in df['dataset'].unique() if d.startswith("LongBench")])

# Plot for WikiText Perplexity (Log Scale)
plt.figure(figsize=(8, 5))
wiki_data = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl']).sort_values('latent_dim')
if not wiki_data.empty:
    raw_ppl_baseline_wiki = wiki_data['raw_ppl'].iloc[0]
    plt.plot(wiki_data['latent_dim'], [raw_ppl_baseline_wiki] * len(wiki_data), linestyle='--', marker='o', label='Raw Baseline')
    plt.plot(wiki_data['latent_dim'], wiki_data['ae_ppl'], marker='x', label='AE')
    plt.xlabel("Latent Dimension")
    plt.ylabel("Perplexity (Log Scale)")
    plt.title("WikiText Perplexity vs. Latent Dimension")
    plt.yscale('log')
    plt.legend()
    plt.grid(True, which="both")
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, "exp1_wikitext_perplexity_vs_latent_dim_logy.png"))
    plt.close()

# Plot for LongBench Perplexity (averaged across tasks, Log Scale)
if longbench_tasks:
    plt.figure(figsize=(10, 6))
    longbench_data = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'])
    if not longbench_data.empty:
        avg_raw_ppl_lb = longbench_data.groupby('latent_dim')['raw_ppl'].mean()
        avg_ae_ppl_lb = longbench_data.groupby('latent_dim')['ae_ppl'].mean()

        plt.plot(avg_raw_ppl_lb.index, avg_raw_ppl_lb.values, linestyle='--', marker='o', label='Avg. Raw Baseline')
        plt.plot(avg_ae_ppl_lb.index, avg_ae_ppl_lb.values, marker='x', label='Avg. AE')
        plt.xlabel("Latent Dimension")
        plt.ylabel("Average Perplexity (LongBench, Log Scale)")
        plt.yscale('log')
        plt.title("Average LongBench Perplexity vs. Latent Dimension")
        plt.legend()
        plt.grid(True, which="both")
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, "exp1_longbench_avg_perplexity_vs_latent_dim_logy.png"))
        plt.close()

# Create Table Data
table_data = []
wiki_data_table = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl']).sort_values('latent_dim')
if not wiki_data_table.empty:
    raw_ppl_baseline_wiki_table = wiki_data_table['raw_ppl'].iloc[0]
    for index, row in wiki_data_table.iterrows():
        table_data.append({
            "Latent Dim": int(row['latent_dim']),
            "Dataset": "WikiText",
            "Raw PPL": f"{raw_ppl_baseline_wiki_table:.2f}",
            "AE PPL": f"{row['ae_ppl']:.2f}"
        })

longbench_data_table = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'])
if not longbench_data_table.empty:
    avg_results_lb = longbench_data_table.groupby('latent_dim').agg(
        Avg_Raw_PPL=('raw_ppl', 'mean'),
        Avg_AE_PPL=('ae_ppl', 'mean')
    ).reset_index()
    for index, row in avg_results_lb.iterrows():
        table_data.append({
            "Latent Dim": int(row['latent_dim']),
            "Dataset": "LongBench (Avg)",
            "Raw PPL": f"{row['Avg_Raw_PPL']:.2f}",
            "AE PPL": f"{row['Avg_AE_PPL']:.2f}"
        })

table_df = pd.DataFrame(table_data)
display(table_df)

# Create LaTeX Table
latex_table = table_df.to_latex(index=False, float_format="%.2f", caption="Perplexity Comparison vs. Latent Dimension (No Compression)", label="tab:perplexity_vs_latent_dim")
print("\nLaTeX Table:")
print(latex_table)

print(f"\nPerplexity vs. Latent Dimension plots (log y-axis) and table (with LaTeX output) saved/printed.")

,Latent Dim,Dataset,Raw PPL,AE PPL
0,8,WikiText,76.19,5049.08
1,16,WikiText,76.19,2192.71
2,32,WikiText,76.19,167.00



LaTeX Table:
\begin{table}
\caption{Perplexity Comparison vs. Latent Dimension (No Compression)}
\label{tab:perplexity_vs_latent_dim}
\begin{tabular}{rlll}
\toprule
Latent Dim & Dataset & Raw PPL & AE PPL \\
\midrule
8 & WikiText & 76.19 & 5049.08 \\
16 & WikiText & 76.19 & 2192.71 \\
32 & WikiText & 76.19 & 167.00 \\
\bottomrule
\end{tabular}
\end{table}


Perplexity vs. Latent Dimension plots (log y-axis) and table (with LaTeX output) saved/printed.


In [22]:
# KV‑cache compression analysis (open‑source only)
# ------------------------------------------------
import json, glob, re, statistics, itertools, os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# Ensure the plots directory exists
plots_dir = "./plots"
os.makedirs(plots_dir, exist_ok=True)

# 1) locate result files in the current directory
files = glob.glob("benchmark_results_distilgpt2_*.json")
if not files:
    raise FileNotFoundError("No benchmark_results_distilgpt2_*.json files in this folder!")

records: List[Dict[str, Any]] = []

# Function to extract latent dimension from filename
def get_latent_dim(filename: str) -> Optional[int]:
    match_ld = re.search(r"distilgpt2_(\d+)\.json$", filename)
    if match_ld:
        return int(match_ld.group(1))
    return None

quantization_levels = [2, 4, 8, 16, 32]

for path in files:
    latent_dim = get_latent_dim(path)
    if latent_dim is not None:
        with open(path) as fp:
            res = json.load(fp)

        raw_ppl: float = res["raw_baseline_ppl"]
        ae_ppl_none: Optional[float] = res["perplexities"].get("none", {}).get("ae_compressed_ppl")
        kv_ppl_quant = {bits: res["perplexities"].get(str(bits), {}).get("kv_cache_baseline_ppl") for bits in quantization_levels}

        if raw_ppl is not None and ae_ppl_none is not None:
            record = {"latent_dim": latent_dim, "raw_ppl": raw_ppl, "ae_ppl": ae_ppl_none, "dataset": "WikiText"}
            record.update({f"raw_quant{b}_ppl": kv_ppl_quant[b] for b in quantization_levels})
            records.append(record)

        lb_baseline = res.get("longbench", {}).get("baseline", {})
        lb_compressed_none = res.get("longbench", {}).get("compressed", {}).get("none", {})
        lb_compressed_quant = {
            bits: res.get("longbench", {}).get("compressed", {}).get(str(bits), {}) for bits in quantization_levels
        }

        for task, base_ppl in lb_baseline.items():
            comp_ppl_none = lb_compressed_none.get(task)
            comp_ppl_quant = {bits: lb_compressed_quant[bits].get(task) for bits in quantization_levels}
            if comp_ppl_none is not None:
                record = {"latent_dim": latent_dim, "raw_ppl": base_ppl, "ae_ppl": comp_ppl_none, "dataset": f"LongBench ({task})"}
                record.update({f"raw_quant{b}_ppl": comp_ppl_quant[b] for b in quantization_levels})
                records.append(record)

df = pd.DataFrame(records).sort_values(["latent_dim", "dataset"]).reset_index(drop=True)

latent_dims = sorted(df['latent_dim'].dropna().unique())
longbench_tasks = sorted([d.split('(')[1][:-1] for d in df['dataset'].unique() if d.startswith("LongBench")])

# Plot for WikiText Perplexity Comparison (Log Scale)
plt.figure(figsize=(10, 6))
wiki_data = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels]).sort_values('latent_dim')
if not wiki_data.empty:
    plt.plot(wiki_data['latent_dim'], wiki_data['raw_ppl'], linestyle='--', marker='o', label='Raw Baseline (32-bit)')
    for bits in quantization_levels:
        plt.plot(wiki_data['latent_dim'], wiki_data[f'raw_quant{bits}_ppl'], marker='v', label=f'Raw ({bits}-bit)')
    plt.plot(wiki_data['latent_dim'], wiki_data['ae_ppl'], marker='x', label='AE (No Comp)')
    plt.xlabel("Latent Dimension")
    plt.ylabel("Perplexity (Log Scale)")
    plt.title("WikiText Perplexity vs. Latent Dimension")
    plt.yscale('log')
    plt.legend()
    plt.grid(True, which="both")
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, "exp1_wikitext_perplexity_vs_latent_dim_all_quant_logy.png"))
    plt.close()

# Plot for LongBench Perplexity Comparison (averaged across tasks, Log Scale)
if longbench_tasks:
    plt.figure(figsize=(12, 7))
    longbench_data = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels])
    if not longbench_data.empty:
        avg_raw_ppl_lb = longbench_data.groupby('latent_dim')['raw_ppl'].mean()
        avg_ae_ppl_lb = longbench_data.groupby('latent_dim')['ae_ppl'].mean()
        avg_raw_quant_ppl_lb = {
            bits: longbench_data.groupby('latent_dim')[f'raw_quant{bits}_ppl'].mean() for bits in quantization_levels
        }

        plt.plot(avg_raw_ppl_lb.index, avg_raw_ppl_lb.values, linestyle='--', marker='o', label='Avg. Raw Baseline (32-bit)')
        for bits in quantization_levels:
            plt.plot(avg_raw_quant_ppl_lb[bits].index, avg_raw_quant_ppl_lb[bits].values, marker='v', label=f'Avg. Raw ({bits}-bit)')
        plt.plot(avg_ae_ppl_lb.index, avg_ae_ppl_lb.values, marker='x', label='Avg. AE (No Comp)')
        plt.xlabel("Latent Dimension")
        plt.ylabel("Average Perplexity (LongBench, Log Scale)")
        plt.yscale('log')
        plt.title("Average LongBench Perplexity vs. Latent Dimension")
        plt.legend()
        plt.grid(True, which="both")
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, "exp1_longbench_avg_perplexity_vs_latent_dim_all_quant_logy.png"))
        plt.close()

# Create Table Data
table_data = []
wiki_data_table = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels]).sort_values('latent_dim')
if not wiki_data_table.empty:
    for index, row in wiki_data_table.iterrows():
        table_row = {"Latent Dim": int(row['latent_dim']), "Dataset": "WikiText", "AE PPL": f"{row['ae_ppl']:.2f}", "Raw (32-bit) PPL": f"{row['raw_ppl']:.2f}"}
        table_row.update({f"Raw ({b}-bit) PPL": f"{row[f'raw_quant{b}_ppl']:.2f}" for b in quantization_levels})
        table_data.append(table_row)

longbench_data_table = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels])
if not longbench_data_table.empty:
    agg_funcs = {'Avg_AE_PPL': ('ae_ppl', 'mean'), 'Avg_Raw_PPL': ('raw_ppl', 'mean')}
    agg_funcs.update({f'Avg_Raw_{b}bit_PPL': (f'raw_quant{b}_ppl', 'mean') for b in quantization_levels})
    avg_results_lb = longbench_data_table.groupby('latent_dim').agg(**agg_funcs).reset_index()
    for index, row in avg_results_lb.iterrows():
        table_row = {"Latent Dim": int(row['latent_dim']), "Dataset": "LongBench (Avg)", "Avg. AE PPL": f"{row['Avg_AE_PPL']:.2f}", "Avg. Raw (32-bit) PPL": f"{row['Avg_Raw_PPL']:.2f}"}
        table_row.update({f"Avg. Raw ({b}-bit) PPL": f"{row[f'Avg_Raw_{b}bit_PPL']:.2f}" for b in quantization_levels})
        table_data.append(table_row)

table_df = pd.DataFrame(table_data)
display(table_df)

# Create LaTeX Table
latex_table = table_df.to_latex(index=False, float_format="%.2f", caption="Perplexity Comparison: AE vs. Raw Quantization Levels", label="tab:perplexity_comparison_all_quant")
print("\nLaTeX Table:")
print(latex_table)

print(f"\nPerplexity comparison plots (log y-axis) and table (with LaTeX output) saved/printed.")

,Latent Dim,Dataset,AE PPL,Raw (32-bit) PPL,Raw (2-bit) PPL,Raw (4-bit) PPL,Raw (8-bit) PPL,Raw (16-bit) PPL
0,8,WikiText,5049.08,76.19,1310.36,104.02,76.30,76.19
1,16,WikiText,2192.71,72.66,1150.16,97.59,72.62,72.65
2,32,WikiText,167.00,72.66,1150.16,97.59,72.62,72.65



LaTeX Table:
\begin{table}
\caption{Perplexity Comparison: AE vs. Raw Quantization Levels}
\label{tab:perplexity_comparison_all_quant}
\begin{tabular}{rlllllll}
\toprule
Latent Dim & Dataset & AE PPL & Raw (32-bit) PPL & Raw (2-bit) PPL & Raw (4-bit) PPL & Raw (8-bit) PPL & Raw (16-bit) PPL \\
\midrule
8 & WikiText & 5049.08 & 76.19 & 1310.36 & 104.02 & 76.30 & 76.19 \\
16 & WikiText & 2192.71 & 72.66 & 1150.16 & 97.59 & 72.62 & 72.65 \\
32 & WikiText & 167.00 & 72.66 & 1150.16 & 97.59 & 72.62 & 72.65 \\
\bottomrule
\end{tabular}
\end{table}


Perplexity comparison plots (log y-axis) and table (with LaTeX output) saved/printed.


In [24]:
# KV‑cache compression analysis (open‑source only)
# ------------------------------------------------
import json, glob, re, statistics, itertools, os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# Ensure the plots directory exists
plots_dir = "./plots"
os.makedirs(plots_dir, exist_ok=True)

# 1) locate result files in the current directory
files = glob.glob("benchmark_results_distilgpt2_*.json")
if not files:
    raise FileNotFoundError("No benchmark_results_distilgpt2_*.json files in this folder!")

records: List[Dict[str, Any]] = []

# Function to extract latent dimension from filename
def get_latent_dim(filename: str) -> Optional[int]:
    match_ld = re.search(r"distilgpt2_(\d+)\.json$", filename)
    if match_ld:
        return int(match_ld.group(1))
    return None

quantization_levels = [2, 4, 8, 16, 32]

for path in files:
    latent_dim = get_latent_dim(path)
    if latent_dim is not None:
        with open(path) as fp:
            res = json.load(fp)

        raw_ppl: float = res["raw_baseline_ppl"]
        ae_ppl_none: Optional[float] = res["perplexities"].get("none", {}).get("ae_compressed_ppl")
        kv_ppl_quant = {bits: res["perplexities"].get(str(bits), {}).get("kv_cache_baseline_ppl") for bits in quantization_levels}

        if raw_ppl is not None and ae_ppl_none is not None:
            record = {"latent_dim": latent_dim, "raw_ppl": raw_ppl, "ae_ppl": ae_ppl_none, "dataset": "WikiText"}
            record.update({f"raw_quant{b}_ppl": kv_ppl_quant[b] for b in quantization_levels})
            records.append(record)

        lb_baseline = res.get("longbench", {}).get("baseline", {})
        lb_compressed_none = res.get("longbench", {}).get("compressed", {}).get("none", {})
        lb_compressed_quant = {
            bits: res.get("longbench", {}).get("compressed", {}).get(str(bits), {}) for bits in quantization_levels
        }

        for task, base_ppl in lb_baseline.items():
            comp_ppl_none = lb_compressed_none.get(task)
            comp_ppl_quant = {bits: lb_compressed_quant[bits].get(task) for bits in quantization_levels}
            if comp_ppl_none is not None:
                record = {"latent_dim": latent_dim, "raw_ppl": base_ppl, "ae_ppl": comp_ppl_none, "dataset": f"LongBench ({task})"}
                record.update({f"raw_quant{b}_ppl": comp_ppl_quant[b] for b in quantization_levels})
                records.append(record)

df = pd.DataFrame(records).sort_values(["latent_dim", "dataset"]).reset_index(drop=True)

latent_dims = sorted(df['latent_dim'].dropna().unique())
longbench_tasks = sorted([d.split('(')[1][:-1] for d in df['dataset'].unique() if d.startswith("LongBench")])

# Plot for WikiText Perplexity Comparison (Log Scale) - Separate Plots
wiki_data = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels]).sort_values('latent_dim')
if not wiki_data.empty:
    for bits in quantization_levels:
        plt.figure(figsize=(8, 5))
        plt.plot(wiki_data['latent_dim'], wiki_data[f'raw_quant{bits}_ppl'], marker='v', label=f'Raw ({bits}-bit)')
        plt.plot(wiki_data['latent_dim'], wiki_data['ae_ppl'], marker='x', label='AE')
        plt.xlabel("Latent Dimension")
        plt.ylabel("Perplexity (Log Scale)")
        plt.title(f"WikiText Perplexity vs. Latent Dim - vs. Raw ({bits}-bit)")
        plt.yscale('log')
        plt.legend()
        plt.grid(True, which="both")
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"exp1_wikitext_perplexity_vs_latent_dim_quant{bits}_logy.png"))
        plt.close()

# Plot for LongBench Perplexity Comparison (averaged across tasks, Log Scale) - Separate Plots
if longbench_tasks:
    longbench_data = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels])
    if not longbench_data.empty:
        avg_ae_ppl_lb = longbench_data.groupby('latent_dim')['ae_ppl'].mean()
        avg_raw_quant_ppl_lb = {
            bits: longbench_data.groupby('latent_dim')[f'raw_quant{bits}_ppl'].mean() for bits in quantization_levels
        }

        for bits in quantization_levels:
            plt.figure(figsize=(10, 6))
            plt.plot(avg_raw_quant_ppl_lb[bits].index, avg_raw_quant_ppl_lb[bits].values, marker='v', label=f'Avg. Raw ({bits}-bit)')
            plt.plot(avg_ae_ppl_lb.index, avg_ae_ppl_lb.values, marker='x', label='Avg. AE')
            plt.xlabel("Latent Dimension")
            plt.ylabel("Average Perplexity (LongBench, Log Scale)")
            plt.title(f"Avg. LongBench PPL vs. Latent Dim - vs. Raw ({bits}-bit)")
            plt.yscale('log')
            plt.legend()
            plt.grid(True, which="both")
            plt.tight_layout()
            plt.savefig(os.path.join(plots_dir, f"exp1_longbench_avg_perplexity_vs_latent_dim_quant{bits}_logy.png"))
            plt.close()

# Create Table Data (as before)
table_data = []
wiki_data_table = df[df['dataset'] == 'WikiText'].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels]).sort_values('latent_dim')
if not wiki_data_table.empty:
    for index, row in wiki_data_table.iterrows():
        table_row = {"Latent Dim": int(row['latent_dim']), "Dataset": "WikiText", "AE PPL": f"{row['ae_ppl']:.2f}"}
        table_row.update({f"Raw ({b}-bit) PPL": f"{row[f'raw_quant{b}_ppl']:.2f}" for b in quantization_levels})
        table_data.append(table_row)

longbench_data_table = df[df['dataset'].str.startswith('LongBench')].dropna(subset=['latent_dim', 'raw_ppl', 'ae_ppl'] + [f'raw_quant{b}_ppl' for b in quantization_levels])
if not longbench_data_table.empty:
    agg_funcs = {'Avg_AE_PPL': ('ae_ppl', 'mean')}
    agg_funcs.update({f'Avg_Raw_{b}bit_PPL': (f'raw_quant{b}_ppl', 'mean') for b in quantization_levels})
    avg_results_lb = longbench_data_table.groupby('latent_dim').agg(**agg_funcs).reset_index()
    for index, row in avg_results_lb.iterrows():
        table_row = {"Latent Dim": int(row['latent_dim']), "Dataset": "LongBench (Avg)", "Avg. AE PPL": f"{row['Avg_AE_PPL']:.2f}"}
        table_row.update({f"Avg. Raw ({b}-bit) PPL": f"{row[f'Avg_Raw_{b}bit_PPL']:.2f}" for b in quantization_levels})
        table_data.append(table_row)

table_df = pd.DataFrame(table_data)
display(table_df)

# Create LaTeX Table (as before)
latex_table = table_df.to_latex(index=False, float_format="%.2f", caption="Perplexity Comparison: AE vs. Raw Quantization Levels", label="tab:perplexity_comparison_all_quant")
print("\nLaTeX Table:")
print(latex_table)

print(f"\nPerplexity comparison plots (log y-axis) and table (with LaTeX output) saved/printed.")

,Latent Dim,Dataset,AE PPL,Raw (2-bit) PPL,Raw (4-bit) PPL,Raw (8-bit) PPL,Raw (16-bit) PPL,Raw (32-bit) PPL
0,8,WikiText,5049.08,1310.36,104.02,76.30,76.19,76.19
1,16,WikiText,2192.71,1150.16,97.59,72.62,72.65,72.66
2,32,WikiText,167.00,1150.16,97.59,72.62,72.65,72.66



LaTeX Table:
\begin{table}
\caption{Perplexity Comparison: AE vs. Raw Quantization Levels}
\label{tab:perplexity_comparison_all_quant}
\begin{tabular}{rlllllll}
\toprule
Latent Dim & Dataset & AE PPL & Raw (2-bit) PPL & Raw (4-bit) PPL & Raw (8-bit) PPL & Raw (16-bit) PPL & Raw (32-bit) PPL \\
\midrule
8 & WikiText & 5049.08 & 1310.36 & 104.02 & 76.30 & 76.19 & 76.19 \\
16 & WikiText & 2192.71 & 1150.16 & 97.59 & 72.62 & 72.65 & 72.66 \\
32 & WikiText & 167.00 & 1150.16 & 97.59 & 72.62 & 72.65 & 72.66 \\
\bottomrule
\end{tabular}
\end{table}


Perplexity comparison plots (log y-axis) and table (with LaTeX output) saved/printed.
